In [1]:
!pip install tensorflow==1.12.0
!pip install coremltools

In [2]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

1.12.0


In [3]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Embedding, Flatten, concatenate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import coremltools

Using TensorFlow backend.


# Model

In [0]:
# Swift for TensorFlow Model

# struct RegressionModel: Module {
#     var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
#     var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
#     var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
#     var embeddingLayer = Dense<Float>(inputSize: (4 + 45), outputSize: 64, activation: relu)
#     var allInputConcatLayer = Dense<Float>(inputSize: (32 + 64), outputSize: 128, activation: relu)
#     var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
#     var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
#     @differentiable
#     func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
#         let numericalInput = numericalLayer(input.numerical[0])
#         let embeddingOutput1 = embedding1(input.categorical[0])
#         let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
#             TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
#         let embeddingOutput2 = embedding2(input.categorical[1])
#         let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
#             TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
#         let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
#         let embeddingInput = embeddingLayer(embeddingConcat)
#         let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
#         return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
#     }
# }

In [0]:
numerical_input = Input(shape=(11,))
categorical_input1 = Input(shape=(1,))
categorical_input2 = Input(shape=(1,))

numerical = Dense(16, activation='relu')(numerical_input)

embedding1 = Embedding(input_dim=2, output_dim=2, input_length=1)(categorical_input1)
embedding1 = Flatten()(embedding1)

embedding2 = Embedding(input_dim=9, output_dim=5, input_length=1)(categorical_input2)
embedding2 = Flatten()(embedding2)

embeddingConcat = concatenate([embedding1, embedding2], axis=1)
embedding = Dense(16, activation='relu')(embeddingConcat)

allConcat = concatenate([numerical, embedding], axis=1)
hidden = Dense(64, activation='relu')(allConcat)
hidden = Dense(32, activation='relu')(hidden)
output = Dense(1)(hidden)

keras_model = Model(inputs=[numerical_input, categorical_input1, categorical_input2], outputs=output)

In [6]:
keras_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 2)         4           input_2[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 5)         45          input_3[0][0]                    
____________________________________________________________________________________________

# Save and Export no Train Model

In [0]:
keras_model.save('keras_no_train_model.h5')

In [8]:
mlmodel = coremltools.converters.keras.convert('keras_no_train_model.h5', use_float_arraytype = True)
mlmodel.save('coreml_no_train.mlmodel')

0 : input_2, <keras.engine.input_layer.InputLayer object at 0x7f947f87a438>
1 : input_3, <keras.engine.input_layer.InputLayer object at 0x7f947f87a4a8>
2 : embedding_1, <keras.layers.embeddings.Embedding object at 0x7f947f87a550>
3 : embedding_2, <keras.layers.embeddings.Embedding object at 0x7f947f87a588>
4 : flatten_1, <keras.layers.core.Flatten object at 0x7f947f87a518>
5 : flatten_2, <keras.layers.core.Flatten object at 0x7f947f87a5c0>
6 : input_1, <keras.engine.input_layer.InputLayer object at 0x7f947f87a6a0>
7 : concatenate_1, <keras.layers.merge.Concatenate object at 0x7f947f87a630>
8 : dense_1, <keras.layers.core.Dense object at 0x7f947f87aac8>
9 : dense_1__activation__, <keras.layers.core.Activation object at 0x7f947ef0c908>
10 : dense_2, <keras.layers.core.Dense object at 0x7f947f87a7b8>
11 : dense_2__activation__, <keras.layers.core.Activation object at 0x7f947ef0c3c8>
12 : concatenate_2, <keras.layers.merge.Concatenate object at 0x7f947f87a940>
13 : dense_3, <keras.layers.c

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
spec = mlmodel.get_spec()
spec

specificationVersion: 1
description {
  input {
    name: "input1"
    type {
      multiArrayType {
        shape: 11
        dataType: FLOAT32
      }
    }
  }
  input {
    name: "input2"
    type {
      multiArrayType {
        shape: 1
        dataType: FLOAT32
      }
    }
  }
  input {
    name: "input3"
    type {
      multiArrayType {
        shape: 1
        dataType: FLOAT32
      }
    }
  }
  output {
    name: "output1"
    type {
      multiArrayType {
        shape: 1
        dataType: FLOAT32
      }
    }
  }
  metadata {
    userDefined {
      key: "coremltoolsVersion"
      value: "3.3"
    }
  }
}
neuralNetwork {
  layers {
    name: "embedding_1"
    input: "input2"
    output: "embedding_1_output"
    embedding {
      inputDim: 2
      outputChannels: 2
      weights {
        floatValue: 0.03467612341046333
        floatValue: -0.04324455186724663
        floatValue: 0.01923951879143715
        floatValue: -0.03662766143679619
      }
    }
  }
  layers {
